In [323]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# to be used a prefix for all files
path_prefix = "/content/drive/My Drive/cs210"


Mounted at /content/drive


In [335]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [336]:
fname = "StockMarket Dataset.csv"
f_name_2="programming language trend over time.csv"
data1 = pd.read_csv(join(path_prefix, fname))
data2=pd.read_csv(join(path_prefix, f_name_2))


Related data connected

In [337]:
data1 = data1[["Date", "Microsoft_Price", "Gold_Price"]]

# Convert 'Date' column to datetime format
data1.loc[:, 'Date'] = pd.to_datetime(data1['Date'], format='%d-%m-%Y')

# Set 'Date' column as the index
data1.set_index('Date', inplace=True)

In [338]:
data2.rename(columns={'Week': 'Date'}, inplace=True)
# Convert 'Date' column to datetime
data2['Date'] = pd.to_datetime(data2['Date'], errors='coerce')

In [339]:
data1= data1.sort_values(by='Date')
data2= data2.sort_values(by='Date')

In [340]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
total_python_per_month = data2.groupby('Date')['Python'].sum().reset_index()
total_python_per_month['Python'] = scaler.fit_transform(total_python_per_month[['Python']])
data1.reset_index(inplace=True)


In [342]:
import pandas as pd

# Assuming 'Date' column is not already in datetime format
data1['Date'] = pd.to_datetime(data1['Date'])

# Set 'Date' column as the index of the DataFrame
data1.set_index('Date', inplace=True)
data1.reset_index(inplace=True)

,Date,Microsoft_Price,Gold_Price
0,2019-02-04,105.74,"1,319.30"
1,2019-02-05,107.22,"1,319.20"
2,2019-02-06,106.03,"1,314.40"
3,2019-02-07,105.27,"1,314.20"
4,2019-02-08,105.67,"1,318.50"
...,...,...,...
1238,2024-01-29,409.72,"2,034.90"
1239,2024-01-30,408.59,"2,050.90"
1240,2024-01-31,397.58,"2,067.40"
1241,2024-02-01,403.78,"2,071.10"


In [343]:
# Assuming 'Date' column is not already in datetime format
data2['Date'] = pd.to_datetime(data2['Date'])

# Set 'Date' column as the index of the DataFrame
data2.set_index('Date', inplace=True)
data2.reset_index(inplace=True)

In [357]:
data1_subset = data1[['Date', 'Microsoft_Price']]
data2_subset = data2[['Date', 'Python']]

In [356]:
# Merge the resampled DataFrames on the 'Date' column
merged_monthly_data = pd.merge(data1_subset,data2_subset, on='Date', how='inner')

# Display the merged data
print(merged_monthly_data.head())

Empty DataFrame
Columns: [Date, Microsoft_Price, Month_x, Python, Month_y]
Index: []


In [418]:
import pandas as pd

# Assuming you have monthly_oil_price and monthly_data2 DataFrames derived from data1_subset and data2_subset respectively

# Extract year and month from 'Date' column for both DataFrames
data1_subset['Year_Month'] = data1_subset['Date'].dt.to_period('M')
data2_subset['Year_Month'] = data2_subset['Date'].dt.to_period('M')

# Merge DataFrames based on 'Year_Month' column
merged_data = pd.merge(data1_subset, data2_subset, on='Year_Month', how='inner')

# Keep only the desired columns
merged_data = merged_data[['Date_x', 'Microsoft_Price', 'Python']]

# Drop duplicates based on 'Date_x'
merged_data = merged_data.drop_duplicates(subset=['Date_x'])

# Convert 'Date_x' to datetime
merged_data['Date_x'] = pd.to_datetime(merged_data['Date_x'])

# Group by year and month and apply a custom function to select five days from each month
selected_dates = merged_data.groupby([merged_data['Date_x'].dt.year, merged_data['Date_x'].dt.month]).head(1)

# Reset index to remove the multi-index
selected_dates.reset_index(drop=True, inplace=True)

# Display the selected dates
print(selected_dates)


       Date_x  Microsoft_Price  Python
0  2019-04-01           119.02      55
1  2019-05-01           127.88      56
2  2019-06-03           119.84      54
3  2019-07-01           135.68      60
4  2019-08-01           138.06      57
5  2019-09-03           136.04      62
6  2019-10-01           137.07      64
7  2019-11-01           143.72      64
8  2019-12-02           149.55      63
9  2020-01-02           160.62      57
10 2020-02-03           174.38      62
11 2020-03-02           172.79      67
12 2020-04-01           152.11      65
13 2020-05-01           174.57      66
14 2020-06-01           182.83      62
15 2020-07-01           204.70      63
16 2020-08-03           216.54      60
17 2020-09-01           227.27      68
18 2020-10-01           212.46      61
19 2020-11-02           202.33      47
20 2020-12-01           216.21      54
21 2021-01-04           217.69      44
22 2021-02-01           239.65      51
23 2021-03-01           236.94      55
24 2021-04-01           2

In [419]:
# Calculate the median oil price
median_price = merged_data['Microsoft_Price'].median()

# Define high and low price groups
high_price_data = merged_data[merged_data['Microsoft_Price'] > median_price]['Python']
low_price_data = merged_data[merged_data['Microsoft_Price'] <= median_price]['Python']


In [420]:
from scipy.stats import ttest_ind

# Perform the t-test
t_stat, p_value = ttest_ind(high_price_data, low_price_data, equal_var=False)  # equal_var=False if variances might be unequal

print("T-Statistic:", t_stat)
print("P-value:", p_value)

T-Statistic: 15.502606118967792
P-value: 2.102833708505945e-49


In [421]:
import statsmodels.api as sm

# Add a constant to the independent variable matrix (required for statsmodels)
X = sm.add_constant(data1_subset['Microsoft_Price'])  # Independent variable (Microsoft price)
Y = data2_subset['Python']                            # Dependent variable (Python)

# Ensure that X and Y have the same number of observations
if X.shape[0] != Y.shape[0]:
    # If not, trim the longer DataFrame to match the length of the shorter one
    min_length = min(X.shape[0], Y.shape[0])
    X = X[:min_length]
    Y = Y[:min_length]

# Reset indices of X and Y to ensure alignment
X.reset_index(drop=True, inplace=True)
Y.reset_index(drop=True, inplace=True)

# Fit the linear regression model
model = sm.OLS(Y, X).fit()

# Print model summary
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                 Python   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.251
Method:                 Least Squares   F-statistic:                     88.53
Date:                Tue, 21 May 2024   Prob (F-statistic):           2.77e-18
Time:                        18:18:33   Log-Likelihood:                -1025.7
No. Observations:                 262   AIC:                             2055.
Df Residuals:                     260   BIC:                             2062.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              13.9581      5.895     

In [422]:
from sklearn.model_selection import train_test_split

X = merged_data[['Microsoft_Price']]  # Features
y = merged_data['Python']     # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [423]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Initialize and train the kNN model
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

# Predict on the test set
knn_predictions = knn.predict(X_test)

# Evaluate the model
knn_mse = mean_squared_error(y_test, knn_predictions)
print(f'kNN Mean Squared Error: {knn_mse}')


kNN Mean Squared Error: 148.1058921161826


In [424]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on the test set
rf_predictions = rf.predict(X_test)


# Evaluate the model
rf_mse = mean_squared_error(y_test, rf_predictions)
print(f'Random Forest Mean Squared Error: {rf_mse}')

Random Forest Mean Squared Error: 203.1789098846266


In [428]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the Random Forest model
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)

# Predict on the test set
rf_predictions = rf.predict(X_test)

# Evaluate the model
rf_mse = mean_squared_error(y_test, rf_predictions)
print(f'Random Forest Mean Squared Error: {rf_mse}')

Random Forest Mean Squared Error: 182.16355976150018


In [426]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=80)

# Initialize and train the kNN model
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_test)
knn_mse = mean_squared_error(y_test, knn_predictions)
print(f'kNN Mean Squared Error: {knn_mse}')

# Initialize and train the Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_predictions = rf.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_predictions)
print(f'Random Forest Mean Squared Error: {rf_mse}')


kNN Mean Squared Error: 156.4516182572614
Random Forest Mean Squared Error: 184.16476288022082
